In [22]:
import json

with open('Data/advanced_fpl_cleaned_data.json', 'r') as json_data:
    data = json.load(json_data)

In [23]:
#Index is (4-player["element_type"]) Forward, midfield, defender, goalie
togga_scoring = {
"goals_scored": [9,9,10,10],
"assists": [6,6,8,9],
"total_through_ball": [2,2,2,6],
"ontarget_scoring_att": [2,2,2,2],
"total_cross": [1,1,1,1],
"successful_dribbles": [1,1,1,1],
"dispossessed": [-0.5,-0.5,-0.5,-0.5],
"own_goals": [-9,-9,-9,-9],
"clean_sheets": [0,1,6,8],
"saves": [0,0,0,2],
"interception": [1,1,1,1],
"tackles": [1,1,1,1],
"goals_conceded": [0,0,-2,-2],
"penalties_saved": [0,0,0,8],
"yellow_cards": [-3,-3,-3,-3],
"red_cards": [-7,-7,-7,-7],
"aerials won": [0.5,0.5,1,1],
"total_clearance": [0,0,0.25,0.25]}

In [24]:
for player in data["elements"]:
    index = 4-player["element_type"]
    for season in player["history"]:
        score = 0
        for stat in togga_scoring:
            if player["history"][season].get(stat):
                score += togga_scoring[stat][index]
        player["history"][season]["togga_score"] = score

In [25]:
with open('Data/example_player.json', 'w') as json_data:
    json.dump(data["elements"][176], json_data)

In [34]:
with open('Data/advanced_fpl_cleaned_data.json', 'w') as json_data:
    json.dump(data, json_data)

In [33]:
one_year = range(683)
two_year = []
three_year = []
four_year = []
five_year = []
for i, player in enumerate(data["elements"]):
    if(len(player["history"])>1):
        two_year.append(i)
    if(len(player["history"])>2):
        three_year.append(i)
    if(len(player["history"])>3):
        four_year.append(i)
    if(len(player["history"])>4):
        five_year.append(i)

TypeError: range(0, 683) is not JSON serializable

SyntaxError: invalid syntax (<ipython-input-32-47fd36de0e8f>, line 1)